#### Recap

For equality-constrained problem

$$\begin{align*}
\min\,\,\, & f(x) \\
\text{s.t. }\,\,\, & Ax= b \\
\end{align*}$$

write out the Lagrangian

$$L(x, \nu)=f(x)+\nu^T(Ax-b)$$

we can do dual (sub)gradient method by starting with some initial $\nu^0$ and repeating

$$\begin{align*}
x^{k+1} &\in \arg \min_x L(x, \nu^k)\\
\nu^{k+1} &= \nu^k+t_k(Ax^{k+1}-b)
\end{align*}$$

#### Augmented Lagrangian method

Augmented Lagrangian method modifies the problem, for $\rho>0$

$$\begin{align*}
\min\,\,\, & f(x)+\frac{\rho}{2}\|Ax-b\|_2^2 \\
\text{s.t. }\,\,\, & Ax= b \\
\end{align*}$$

Using a modified Lagrangian

$$L_{\rho}(x, \nu)=f(x)+\nu^T(Ax-b)+\frac{\rho}{2}\|Ax-b\|_2^2$$

we get the iterates

$$\begin{align*}
x^{k+1} &\in \arg \min_x L_{\rho}(x, \nu^k)\\
\nu^{k+1} &= \nu^k+\rho(Ax^{k+1}-b)
\end{align*}$$

Notice the step size choice $t_k=\rho$

Since $x^{k+1}$ minimizes $f(x)+\left(\nu^k\right)^TAx+(\rho/2)\|Ax-b\|_2^2$ over $x$, we have

$$\begin{align*}
0&\in \partial f(x^{k+1})+A^T\left(\nu^k+\rho(Ax^{k+1}-b)\right) \\
&=\partial f(x^{k+1})+A^T\nu^{k+1}
\end{align*}$$

This is the `stationary condition` for the primal problem, so the KKT conditions are satisfied in the limit